# Day 38

I'm adding onto yesterday's query by filtering for number 1 overall draft picks that finished outside the 1st round. I bet most of those finishes are due to injury so I will also join against the injury table I have from the nflverse data in my database.

In [1]:
import pandas as pd
import sqlite3

# Create database connection
conn = sqlite3.connect('../../data/db/database.db')

## Query the Data

In [2]:
query = """
WITH season_summary AS (
    SELECT
        player_id,
        player_display_name AS name,
        season,
        position,
        -- Fix team to match ADP table
        CASE
            WHEN recent_team = 'LA' THEN 'LAR'
            ELSE recent_team
            END AS team,
        ROUND(SUM(fantasy_points),2) AS tot_pts,
        ROUND(SUM(fantasy_points_ppr),2) AS tot_pts_ppr
    FROM weekly
    WHERE season_type = 'REG' 
        AND player_display_name IS NOT NULL
    GROUP BY player_id, player_display_name, season),
rankings AS (
    SELECT
        *,
        RANK() OVER(PARTITION BY season ORDER BY tot_pts DESC) AS r_pts,
        RANK() OVER(PARTITION BY season ORDER BY tot_pts_ppr DESC) AS r_pts_ppr
    FROM season_summary), 
joined AS (   
    SELECT
        adp.*, 
        rankings.*, 
        weeks_injured
    FROM adp
    LEFT JOIN rankings
        ON rankings.name = adp.name
        AND rankings.season = adp.season
        AND rankings.position = adp.position
        -- Need to add in case multiple players with same name
        AND rankings.team = adp.team
    LEFT JOIN (
        SELECT *, COUNT(*) AS weeks_injured
        FROM injuries
        WHERE practice_status LIKE 'OUT%'
            OR practice_status LIKE 'Did Not Participate%'
        GROUP BY season, full_name, team, position
    ) AS injuries
        ON injuries.season = adp.season
        AND injuries.full_name = adp.name
        AND injuries.team = adp.team
        AND injuries.position = adp.position),
first_round_picks AS (
    SELECT
       scoring,
        season,
        teams,
        name,
        position,
        team,
        adp_formatted,
        tot_pts_ppr,
        r_pts,
        weeks_injured
    FROM joined
    WHERE round = 1
        AND pick = 1),
standard_under_achievers AS (
    SELECT *
    FROM first_round_picks
    WHERE scoring = 'standard'
        -- AND weeks_injured IS NULL
        AND r_pts > teams),
ppr_under_achievers AS (
    SELECT *
    FROM first_round_picks
    WHERE scoring = 'ppr'
        -- AND weeks_injured IS NULL
        AND r_pts > teams)
SELECT * 
FROM ppr_under_achievers
"""

df = pd.read_sql(query, conn)
df

,scoring,season,teams,name,position,team,adp_formatted,tot_pts_ppr,r_pts,weeks_injured
0,ppr,2010,10,Chris Johnson,RB,TEN,1.01,272.9,17,NaN
1,ppr,2012,10,Arian Foster,RB,HOU,1.01,302.1,14,1.0
2,ppr,2013,10,Adrian Peterson,RB,MIN,1.01,232.7,34,4.0
3,ppr,2016,10,Antonio Brown,WR,PIT,1.01,307.3,35,1.0
4,ppr,2017,10,David Johnson,RB,ARI,1.01,13.0,429,NaN
5,ppr,2019,10,Saquon Barkley,RB,NYG,1.01,244.1,37,1.0
6,ppr,2020,10,Christian McCaffrey,RB,CAR,1.01,90.4,171,6.0
7,ppr,2021,10,Christian McCaffrey,RB,CAR,1.01,127.5,139,2.0
8,ppr,2010,12,Chris Johnson,RB,TEN,1.01,272.9,17,NaN
9,ppr,2012,12,Arian Foster,RB,HOU,1.01,302.1,14,1.0


Notes:
- David Johnson had a season-ending injury but never showed up in the injury table
- Christian McCaffrey was on IR for the two seasons on this list. I don't believe that shows up in the injury table 

I'm going to filter out these outliers...

In [3]:
df.query("r_pts < 50")

,scoring,season,teams,name,position,team,adp_formatted,tot_pts_ppr,r_pts,weeks_injured
0,ppr,2010,10,Chris Johnson,RB,TEN,1.01,272.9,17,NaN
1,ppr,2012,10,Arian Foster,RB,HOU,1.01,302.1,14,1.0
2,ppr,2013,10,Adrian Peterson,RB,MIN,1.01,232.7,34,4.0
3,ppr,2016,10,Antonio Brown,WR,PIT,1.01,307.3,35,1.0
5,ppr,2019,10,Saquon Barkley,RB,NYG,1.01,244.1,37,1.0
8,ppr,2010,12,Chris Johnson,RB,TEN,1.01,272.9,17,NaN
9,ppr,2012,12,Arian Foster,RB,HOU,1.01,302.1,14,1.0
10,ppr,2013,12,Adrian Peterson,RB,MIN,1.01,232.7,34,4.0
11,ppr,2016,12,Antonio Brown,WR,PIT,1.01,307.3,35,1.0
13,ppr,2019,12,Saquon Barkley,RB,NYG,1.01,244.1,37,1.0


These players still finished within the top 40 for total fantasy points which is pretty good, with almost every player having at least 1 week of an injury.  

From my analysis the past 3 days, it looks to me that first round players are consistently good, but won't win you your league. Getting the #1 pick and picking Christian McCaffrey isn't going to do you any good if he gets injured and if he doesn't odds are he won't be a bust. None of these players will be. 

I want to follow this up with the following: what is the average drop off in fantasy points between the number 1 overall pick and the last pick in the first round? Or the last pick of the second? In snake drafts the manager with the #1 pick will pick last in the 2nd. We know that the #1 pick can sometimes finish 14-37? My guess is that it's actually better to pick middle to late 1st round so that you can get two solid players.